In [ ]:
# Step 1: Load image data from CSV file

import os
import pandas as pd
import numpy as np
from PIL import Image

def load_image_data(data_path):
    base_path = '/content/drive/MyDrive/autism-data/autism/'
    df = pd.read_csv(data_path)

    # Load training data
    train_df = df[df['dataset'] == 'train']
    image_list_train = list(train_df['filepaths'])
    imgs = []
    imgs_not_found = []
    for img in image_list_train:
        path = os.path.join(base_path, img)
        if os.path.exists(path):
            image = Image.open(path)
            image = image.resize((256, 256))
            imgs.append(np.array(image))
        else:
            imgs_not_found.append(img)
    for img in imgs_not_found:
        train_df = train_df[train_df['filepaths'] != img]
    X_train = np.array(imgs)
    y_train = train_df['labels'].apply(lambda x: 1 if x == 'autistic' else 0)

    # Load testing data
    test_df = df[df['dataset'] == 'test']
    image_list_test = list(test_df['filepaths'])
    imgs_test = []
    imgs_test_missing = []
    for img in image_list_test:
        path = os.path.join(base_path, img)
        if os.path.exists(path):
            image = Image.open(path)
            image = image.resize((256, 256))
            imgs_test.append(np.array(image))
        else:
            imgs_test_missing.append(img)
    for img in imgs_test_missing:
        test_df = test_df[test_df['filepaths'] != img]
    X_test = np.array(imgs_test)
    y_test = test_df['labels'].apply(lambda x: 1 if x == 'autistic' else 0)

    # Load validation data
    val_df = df[df['dataset'] == 'valid']
    image_list_val = list(val_df['filepaths'])
    imgs_val = []
    imgs_val_missing = []
    for img in image_list_val:
        path = os.path.join(base_path, img)
        if os.path.exists(path):
            image = Image.open(path)
            image = image.resize((256, 256))
            imgs_val.append(np.array(image))
        else:
            imgs_val_missing.append(img)
    for img in imgs_val_missing:
        val_df = val_df[val_df['filepaths'] != img]
    X_val = np.array(imgs_val)
    y_val = val_df['labels'].apply(lambda x: 1 if x == 'autistic' else 0)

    return X_train, y_train, X_test, y_test, X_val, y_val


In [ ]:
# Step 2: Define the model architecture

import tensorflow as tf
from tensorflow.keras import Sequential, layers
from tensorflow.keras.layers.experimental.preprocessing import Rescaling

def load_own_model():
    model = Sequential()
    model.add(Rescaling(1./255, input_shape=(256, 256, 3)))

    model.add(layers.Conv2D(16, kernel_size=10, activation='relu'))
    model.add(layers.MaxPooling2D(3))

    model.add(layers.Conv2D(32, kernel_size=8, activation="relu"))
    model.add(layers.MaxPooling2D(3))

    model.add(layers.Conv2D(32, kernel_size=6, activation="relu"))
    model.add(layers.Conv2D(32, kernel_size=6, activation="relu"))
    model.add(layers.MaxPooling2D(3))

    return model

In [ ]:
# Step 3: Train the model

from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping

def train_model(model, X_train, y_train, X_val, y_val):
    model.compile(optimizer=optimizers.Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(patience=5, restore_best_weights=True)
    history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=20, batch_size=32, callbacks=[early_stopping])
    return model, history

In [ ]:
# Step 4: Perform facial assessment using the trained model

import base64
import requests
import streamlit as st

def perform_facial_assessment(image):
    # Convert image to base64
    image_base64 = base64.b64encode(image).decode('utf-8')

    # Send image to the API for preprocessing and prediction
    api_url = 'https://your-api-url/predict'
    headers = {'Content-Type': 'application/json'}
    payload = {'image': image_base64}
    response = requests.post(api_url, headers=headers, json=payload)

    if response.status_code == 200:
        # Get the prediction result from the response
        result = response.json()['result']
        probability = response.json()['probability']

        # Display the result
        st.subheader('Detection Result:')
        st.success(f'Likelihood of being autistic: {probability:.2f}')
        st.info(f'Classification: {result}')
    else:
        st.error('Error occurred while processing the image.')

    # Display the uploaded image
    st.image(image, caption='Uploaded Image', use_column_width=True)

In [ ]:
# Step 5: Create the Streamlit app

 def main():
    st.set_page_config(
        page_title="Facial Assessment Tool",
        page_icon="🧩",
        layout="centered",
        initial_sidebar_state="collapsed",
    )

    st.title('Facial Assessment Tool')

    # Display the introduction and instructions
    st.markdown('''
        ## Facial Assessment Tool
        Upload an image of a child to assess the likelihood of autism based on facial morphology.
        ''')

    # Upload image
    uploaded_file = st.file_uploader('Upload an image', type=['jpg', 'jpeg', 'png'])

    if uploaded_file is not None:
        # Read image file
        image = uploaded_file.read()

        # Perform facial assessment using the trained model
        perform_facial_assessment(image)

    # Citation
    st.markdown('''
        This tool is based on research papers conducted by Naomi Scott, Alex Lee Jones, Robin Stewart Samuel Kramer, Robert Ward, Mohammad-Parsa Hosseini, Madison Beary, Alex Hadsell, 
        Ryan Messersmith, Hamid Soltanian-Zadeh, K.K. Mujeeb Rahman and M. Monica Subashini. You can find the studies at the following links:
        
        - [Bangor University Study](https://ward-lab.bangor.ac.uk/pubs/Scott_Ward_14_AQ.pdf)
        - [Deep Learning for Autism Diagnosis and Facial Analysis in Children](https://www.frontiersin.org/articles/10.3389/fncom.2021.789998/full)
        - [Identification of Autism in Children Using Static Facial Features and Deep Neural Networks](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8773918/)
        
        Please note that this tool is provided for informational purposes only and is not a diagnostic tool. It assesses the likelihood of autism based on facial morphology, but a formal diagnosis should be made by a qualified healthcare professional.
        ''')

In [ ]:
if __name__ == '__main__':
    # Step 6: Load image data
    data_path = '/content/drive/MyDrive/autism-data/autism.csv'
    X_train, y_train, X_test, y_test, X_val, y_val = load_image_data()

    # Step 7: Load the model
    model = load_own_model()

    # Step 8: Train the model
    trained_model, history = train_model(model, X_train, y_train, X_val, y_val)

    # Step 9: Run the Streamlit app
    main()